<a href="https://colab.research.google.com/github/analyticsariel/projects/blob/master/Estated_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estated Property Data

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com |
| External References   | API |
| Input Datasets        | Source name |
| Output Datasets       | Source name |
| Input Data Source     | Pandas DataFrame |
| Output Data Source    | Pandas DataFrame |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 24th Jan 2022 | Ariel Herrera | Create notebook. |

## Getting Started
1. Copy this notebook -> File -> Save a Copy in Drive
2. Directions

## Useful Resources
- [Google Colab Cheat Sheet](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)
- [Estated Data Dictionary](https://estated.com/developers/docs/v4/property/bulk)
- [Estated Data Schema](https://estated.com/developers/docs/v4/property/schema)
- [How to convert JSON into a Pandas DataFrame](https://towardsdatascience.com/how-to-convert-json-into-a-pandas-dataframe-100b2ae1e0d8)
- [Convert CURL request to Python](https://curlconverter.com/#)

## Install Packages

## Imports

In [ ]:
from google.colab import drive, files # google colab specific
import requests
import pandas as pd
import os
import warnings

## Functions

In [ ]:
def get_estated_detail(api_key, 
                       street_address, 
                       city, 
                       state, 
                       zip_code):
  # sample API get request
  base_url = 'https://apis.estated.com/v4/property'

  params = (
      ('token', estated_api_key),
      ('street_address', street_address),
      ('city', city),
      ('state', state),
      ('zip_code', zip_code),
  )

  response = requests.get(base_url, params=params)
  return response

## Locals & Constants

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# set working directory
os.chdir('/content/drive/MyDrive/Colab Data/')
dir = os.getcwd()
print('Current working directory:', dir)

# remove pandas warnings
warnings.filterwarnings('ignore')

Current working directory: /content/drive/MyDrive/Colab Data


## Data

In [ ]:
# read in api keys
df_api_keys = pd.read_csv(dir + '/input/api_keys.csv')
# extract api key
estated_api_key = df_api_keys.loc[df_api_keys['API'] == 'estated']['KEY'].iloc[0]

## Transformations

### <font color="green">Sample API Request - Single Address</font>

In [ ]:
# sample API get request
base_url = 'https://apis.estated.com/v4/property'

params = (
    ('token', estated_api_key),
    ('street_address', '151 Battle Green Dr'),
    ('city', 'Rochester'),
    ('state', 'NY'),
    ('zip_code', '14624'),
)

response = requests.get(base_url, params=params)
print(response.text)

{"data":{"metadata":{"publishing_date":"2021-02-01"},"address":{"street_number":"151","street_pre_direction":null,"street_name":"BATTLE GREEN","street_suffix":"DR","street_post_direction":null,"unit_type":null,"unit_number":null,"formatted_street_address":"151 BATTLE GREEN DR","city":"ROCHESTER","state":"NY","zip_code":"14624","zip_plus_four_code":"4969","carrier_code":"C006","latitude":43.108222,"longitude":-77.704543,"geocoding_accuracy":"PARCEL CENTROID","census_tract":"360550145.043011"},"parcel":{"apn_original":"262200 147.09-2-6","apn_unformatted":"2622001470926","apn_previous":null,"fips_code":"36055","frontage_ft":75.0,"depth_ft":125.0,"area_sq_ft":9583,"area_acres":0.22,"county_name":"MONROE","county_land_use_code":"210","county_land_use_description":"1-FAMILY YEAR-ROUND RESIDENCE","standardized_land_use_category":"RESIDENTIAL","standardized_land_use_type":"SINGLE FAMILY RESIDENTIAL","location_descriptions":[],"zoning":"02 - SINGLE RESIDENCE","building_count":null,"tax_account

In [ ]:
# transform to json response
response_json = response.json()
response_json

{'data': {'address': {'carrier_code': 'C006',
   'census_tract': '360550145.043011',
   'city': 'ROCHESTER',
   'formatted_street_address': '151 BATTLE GREEN DR',
   'geocoding_accuracy': 'PARCEL CENTROID',
   'latitude': 43.108222,
   'longitude': -77.704543,
   'state': 'NY',
   'street_name': 'BATTLE GREEN',
   'street_number': '151',
   'street_post_direction': None,
   'street_pre_direction': None,
   'street_suffix': 'DR',
   'unit_number': None,
   'unit_type': None,
   'zip_code': '14624',
   'zip_plus_four_code': '4969'},
  'assessments': [{'improvement_value': 158300,
    'land_value': 31400,
    'total_value': 189700,
    'year': 2020},
   {'improvement_value': 158300,
    'land_value': 31400,
    'total_value': 189700,
    'year': 2019},
   {'improvement_value': 127900,
    'land_value': 31400,
    'total_value': 159300,
    'year': 2018}],
  'boundary': {'geojson': {'coordinates': [[[[-77.70481631402163,
        43.10823905468868],
       [-77.70466049568529, 43.1084015450

In [ ]:
response_json['data'].keys()

dict_keys(['metadata', 'address', 'parcel', 'boundary', 'structure', 'valuation', 'taxes', 'assessments', 'market_assessments', 'owner', 'deeds'])

In [ ]:
# normalize into a tabular format
df = pd.json_normalize(
    data=response_json['data']
    )
df.head()

,taxes,assessments,market_assessments,deeds,metadata.publishing_date,address.street_number,address.street_pre_direction,address.street_name,address.street_suffix,address.street_post_direction,address.unit_type,address.unit_number,address.formatted_street_address,address.city,address.state,address.zip_code,address.zip_plus_four_code,address.carrier_code,address.latitude,address.longitude,address.geocoding_accuracy,address.census_tract,parcel.apn_original,parcel.apn_unformatted,parcel.apn_previous,parcel.fips_code,parcel.frontage_ft,parcel.depth_ft,parcel.area_sq_ft,parcel.area_acres,parcel.county_name,parcel.county_land_use_code,parcel.county_land_use_description,parcel.standardized_land_use_category,parcel.standardized_land_use_type,parcel.location_descriptions,parcel.zoning,parcel.building_count,parcel.tax_account_number,parcel.legal_description,...,structure.pool_type,structure.architecture_type,structure.construction_type,structure.exterior_wall_type,structure.foundation_type,structure.roof_material_type,structure.roof_style_type,structure.heating_type,structure.heating_fuel_type,structure.air_conditioning_type,structure.fireplaces,structure.basement_type,structure.quality,structure.condition,structure.flooring_types,structure.plumbing_fixtures_count,structure.interior_wall_type,structure.water_type,structure.sewer_type,structure.total_area_sq_ft,structure.other_areas,structure.other_rooms,structure.other_features,structure.other_improvements,structure.amenities,valuation.value,valuation.high,valuation.low,valuation.forecast_standard_deviation,valuation.date,owner.name,owner.second_name,owner.unit_type,owner.unit_number,owner.formatted_street_address,owner.city,owner.state,owner.zip_code,owner.zip_plus_four_code,owner.owner_occupied
0,"[{'year': 2021, 'amount': 8868, 'exemptions': ...","[{'year': 2020, 'land_value': 31400, 'improvem...","[{'year': 2020, 'land_value': 32708, 'improvem...","[{'document_type': 'WARRANTY DEED', 'recording...",2021-02-01,151,None,BATTLE GREEN,DR,None,None,None,151 BATTLE GREEN DR,ROCHESTER,NY,14624,4969,C006,43.108222,-77.704543,PARCEL CENTROID,360550145.043011,262200 147.09-2-6,2622001470926,None,36055,75.0,125.0,9583,0.22,MONROE,210,1-FAMILY YEAR-ROUND RESIDENCE,RESIDENTIAL,SINGLE FAMILY RESIDENTIAL,[],02 - SINGLE RESIDENCE,None,COM-0003926,LOT 27 3000 ACRE TRACT LEXINGTON SUB SEC 12 L-...,...,None,COLONIAL,None,ALUMINUM OR VINYL SIDING,None,None,None,FORCED AIR UNIT,GAS,CENTRAL,1,FULL BASEMENT,C,GOOD,[],None,None,MUNICIPAL,MUNICIPAL,1975,"[{'type': '1ST FLOOR', 'sq_ft': '912'}, {'type...",[RECREATION ROOM],[],"[{'type': 'SHED', 'sq_ft': '10X12'}]",[],249000,273900,224100,10,2022-01-06,KRISTIN N RIMES,None,None,None,151 BATTLE GREEN DR,ROCHESTER,NY,14624,4969,YES


In [ ]:
# download
df.to_csv('estated_output.csv', index=False)
files.download('estated_output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### <font color="green">Sub Tables</font>

In [ ]:
# select keys to view sub tables
print('Keys available under <data>: {}', response_json['data'].keys())
deeds_dict = response_json['data']['deeds']
deeds_dict

Keys available under <data>: {} dict_keys(['metadata', 'address', 'parcel', 'boundary', 'structure', 'valuation', 'taxes', 'assessments', 'market_assessments', 'owner', 'deeds'])


[{'buyer2_first_name': None,
  'buyer2_last_name': None,
  'buyer_address': '151 BATTLE GREEN DR',
  'buyer_city': 'ROCHESTER',
  'buyer_first_name': 'KRISTIN',
  'buyer_last_name': 'RIMES',
  'buyer_state': 'NY',
  'buyer_unit_number': None,
  'buyer_unit_type': None,
  'buyer_zip_code': '14624',
  'buyer_zip_plus_four_code': '4969',
  'deed_book': '12048',
  'deed_page': '266',
  'distressed_sale': False,
  'document_id': '201807030288',
  'document_type': 'WARRANTY DEED',
  'lender_name': None,
  'lender_type': None,
  'loan_amount': None,
  'loan_due_date': None,
  'loan_finance_type': None,
  'loan_interest_rate': None,
  'loan_type': None,
  'original_contract_date': '2018-06-28',
  'real_estate_owned': 'NO',
  'recording_date': '2018-07-03',
  'sale_price': 199000,
  'sale_price_description': 'FULL AMOUNT COMPUTED FROM TRANSFER TAX OR EXCISE TAX',
  'seller2_first_name': None,
  'seller2_last_name': None,
  'seller_address': None,
  'seller_city': None,
  'seller_first_name': 'N

In [ ]:
# view sub tables
df_deeds = pd.DataFrame(deeds_dict)
df_deeds

,document_type,recording_date,original_contract_date,deed_book,deed_page,document_id,sale_price,sale_price_description,transfer_tax,distressed_sale,real_estate_owned,seller_first_name,seller_last_name,seller2_first_name,seller2_last_name,seller_address,seller_unit_number,seller_city,seller_state,seller_zip_code,seller_zip_plus_four_code,buyer_first_name,buyer_last_name,buyer2_first_name,buyer2_last_name,buyer_address,buyer_unit_type,buyer_unit_number,buyer_city,buyer_state,buyer_zip_code,buyer_zip_plus_four_code,lender_name,lender_type,loan_amount,loan_type,loan_due_date,loan_finance_type,loan_interest_rate
0,WARRANTY DEED,2018-07-03,2018-06-28,12048,266,201807030288,199000.0,FULL AMOUNT COMPUTED FROM TRANSFER TAX OR EXCI...,796.0,False,NO,NICHOLAS M,BELLUCCIO,None,None,None,None,None,None,None,None,KRISTIN,RIMES,None,None,151 BATTLE GREEN DR,None,None,ROCHESTER,NY,14624,4969,None,None,NaN,None,None,None,None
1,WARRANTY DEED,2017-06-06,2017-04-04,11870,262,201706060753,168750.0,SALES PRICE OR TRANSFER TAX ROUNDED BY COUNTY ...,675.0,False,NO,CHRISTOPHER M,CROWLEY,None,None,151 BATTLE GREEN DR,None,ROCHESTER,NY,14624,4969,NICHOLAS M,BELLUCCIO,None,None,151 BATTLE GREEN DR,None,None,ROCHESTER,NY,14624,4969,None,None,NaN,None,None,None,None
2,WARRANTY DEED,2016-05-25,2016-05-17,11701,490,201605250391,160750.0,SALES PRICE OR TRANSFER TAX ROUNDED BY COUNTY ...,643.0,False,NO,ELMAR,LOCKER,None,LOCKER FAMILY DYNASTY TRUST,4571 RTE 89,None,SENECA FALLS,NY,13148,None,CHRISTOPHER M,CROWLEY,None,None,1854 CONCORDIA LAKE CIR,UNIT,808,CAPE CORAL,FL,33909,9075,None,None,NaN,None,None,None,None
3,WARRANTY DEED,2013-10-11,2013-10-07,11315,315,201310110734,155000.0,FULL AMOUNT STATED ON DOCUMENT,620.0,False,NO,SONIA M,PALERMO,None,None,151 BATTLE GREEN DR,None,ROCHESTER,NY,14624,4969,ELMAR,LOCKER,ROBERT D,SCOLARO,4571 RT 89,None,None,SENECA FALLS,NY,13149,0000,None,None,NaN,None,None,None,None
4,INTRAFAMILY TRANSFER AND DISSOLUTION,2010-09-27,2010-09-16,10926,081,201009270868,NaN,NON-ARMS LENGTH TRANSACTION,NaN,False,NO,CHRISTOPHER M,PALERMO,SONIA M,PALERMO,151 BATTLE GREEN DR,None,ROCHESTER,NY,14624,4969,SONIA M,PALERMO,None,None,151 BATTLE GREEN DR,None,None,ROCHESTER,NY,14624,4969,RBS CITIZENS NA,BANK,110000.0,NEW CONVENTIONAL,2040-10-01,None,None


### <font color="green">Case Study: Find Potential Investments</font>

In [ ]:
# create a dataframe
d = {'address': ['9210 Horizon Dr, Spring Hill, FL 34608', '11230 Holbrook St, Spring Hill, FL 34609']}
_df_addresses = pd.DataFrame(data=d)
_df_addresses

,address
0,"9210 Horizon Dr, Spring Hill, FL 34608"
1,"11230 Holbrook St, Spring Hill, FL 34609"


In [ ]:
# split dataframe into sub address elements
df_addresses = _df_addresses.copy()
df_addresses['street'] = df_addresses.apply(lambda x: x['address'].split(',')[0], axis=1)
df_addresses['city'] = df_addresses.apply(lambda x: x['address'].split(',')[1].strip(), axis=1)
df_addresses['state'] = df_addresses.apply(lambda x: x['address'].split(',')[2].split()[0], axis=1)
df_addresses['zip_code'] = df_addresses.apply(lambda x: x['address'].split(',')[2].split()[1], axis=1)
df_addresses

,address,street,city,state,zip_code
0,"9210 Horizon Dr, Spring Hill, FL 34608",9210 Horizon Dr,Spring Hill,FL,34608
1,"11230 Holbrook St, Spring Hill, FL 34609",11230 Holbrook St,Spring Hill,FL,34609


In [ ]:
# get estated property detail per address

response_list = []
# iterate through rows in the dataframe (table)
for index, row in df_addresses.iterrows():
  # get parameters
  address = row['address']
  street = row['street']
  city = row['city']
  state = row['state']
  zip_code = row['zip_code']
  # get api response
  print('Getting data for address: {0}'.format(address))
  response = get_estated_detail(estated_api_key, 
                        street, 
                        city, 
                        state, 
                        zip_code)
  # append to list
  response_list.append(response)

Getting data for address: 9210 Horizon Dr, Spring Hill, FL 34608
Getting data for address: 11230 Holbrook St, Spring Hill, FL 34609


In [ ]:
print('Num of responses:', len(response_list))
print('Data keys:', response_list[1].json()['data'])
response_list[1].json()['data']['structure']

Num of responses: 2
Data keys: {'metadata': {'publishing_date': '2021-11-01'}, 'address': {'street_number': '11230', 'street_pre_direction': None, 'street_name': 'HOLBROOK', 'street_suffix': 'ST', 'street_post_direction': None, 'unit_type': None, 'unit_number': None, 'formatted_street_address': '11230 HOLBROOK ST', 'city': 'SPRING HILL', 'state': 'FL', 'zip_code': '34609', 'zip_plus_four_code': '3843', 'carrier_code': 'R029', 'latitude': 28.465462, 'longitude': -82.536269, 'geocoding_accuracy': 'PARCEL CENTROID', 'census_tract': '120530411.062047'}, 'parcel': {'apn_original': 'R32-323-17-5090-0492-0070', 'apn_unformatted': 'R3232317509004920070', 'apn_previous': None, 'fips_code': '12053', 'frontage_ft': None, 'depth_ft': None, 'area_sq_ft': 12188, 'area_acres': 0.28, 'county_name': 'HERNANDO', 'county_land_use_code': '01', 'county_land_use_description': 'SINGLE FAMILY RESIDENCE', 'standardized_land_use_category': 'RESIDENTIAL', 'standardized_land_use_type': 'SINGLE FAMILY RESIDENTIAL'

{'air_conditioning_type': 'YES',
 'amenities': [],
 'architecture_type': None,
 'basement_type': None,
 'baths': 2.0,
 'beds_count': 3,
 'condition': 'AVERAGE',
 'construction_type': 'WOOD',
 'effective_year_built': 1989,
 'exterior_wall_type': 'STUCCO',
 'fireplaces': None,
 'flooring_types': ['COMBINATION'],
 'foundation_type': 'CONCRETE',
 'heating_fuel_type': None,
 'heating_type': 'YES',
 'interior_wall_type': 'GYPSUM BOARD',
 'other_areas': [{'sq_ft': '1820', 'type': 'BASE AREA'},
  {'sq_ft': '176', 'type': 'ENCLOSED PORCH'},
  {'sq_ft': '280', 'type': 'COVERED PORCH'},
  {'sq_ft': '483', 'type': 'MEZZANINE'},
  {'sq_ft': '88', 'type': 'ENCLOSED PORCH'},
  {'sq_ft': '30', 'type': 'COVERED PORCH'},
  {'sq_ft': '16', 'type': 'COVERED PORCH'}],
 'other_features': [{'sq_ft': '589', 'type': 'SCREENED ENCLOSURE'},
  {'sq_ft': '934', 'type': 'CONCRETE PAVING'}],
 'other_improvements': [{'sq_ft': '288', 'type': 'POOL SIZE'}],
 'other_rooms': [],
 'parking_spaces_count': 2,
 'parking_type

In [ ]:
# Python code to merge dict using a single
# expression
def Merge(dict1, dict2, dict3):
    res = {**dict1, **dict2, **dict3}
    return res

structure_data_list = [Merge(resp.json()['data']['address'], resp.json()['data']['valuation'], resp.json()['data']['structure']) for resp in response_list]
pd.DataFrame(structure_data_list)

,street_number,street_pre_direction,street_name,street_suffix,street_post_direction,unit_type,unit_number,formatted_street_address,city,state,zip_code,zip_plus_four_code,carrier_code,latitude,longitude,geocoding_accuracy,census_tract,value,high,low,forecast_standard_deviation,date,year_built,effective_year_built,stories,rooms_count,beds_count,baths,partial_baths_count,units_count,parking_type,parking_spaces_count,pool_type,architecture_type,construction_type,exterior_wall_type,foundation_type,roof_material_type,roof_style_type,heating_type,heating_fuel_type,air_conditioning_type,fireplaces,basement_type,quality,condition,flooring_types,plumbing_fixtures_count,interior_wall_type,water_type,sewer_type,total_area_sq_ft,other_areas,other_rooms,other_features,other_improvements,amenities
0,9210,None,HORIZON,DR,None,None,None,9210 HORIZON DR,SPRING HILL,FL,34608,5530,R060,28.458464,-82.570268,PARCEL CENTROID,120530411.041020,242000,261360,222640,8,2022-01-05,1980,NaN,1,5,3,2.0,None,None,GARAGE,1,None,None,CONCRETE,STUCCO,CONCRETE,TILE,GABLE OR HIP,YES,None,YES,None,None,None,AVERAGE,[CARPET],6,GYPSUM BOARD,None,None,1418,"[{'type': 'BASE AREA', 'sq_ft': '1418'}, {'typ...",[],"[{'type': 'SCREENED ENCLOSURE', 'sq_ft': '840'...",[],[]
1,11230,None,HOLBROOK,ST,None,None,None,11230 HOLBROOK ST,SPRING HILL,FL,34609,3843,R029,28.465462,-82.536269,PARCEL CENTROID,120530411.062047,249000,268920,229080,8,2022-01-05,1983,1989.0,1,5,3,2.0,None,None,GARAGE,2,YES,None,WOOD,STUCCO,CONCRETE,COMPOSITION SHINGLE,GABLE OR HIP,YES,None,YES,None,None,None,AVERAGE,[COMBINATION],6,GYPSUM BOARD,None,None,1820,"[{'type': 'BASE AREA', 'sq_ft': '1820'}, {'typ...",[],"[{'type': 'SCREENED ENCLOSURE', 'sq_ft': '589'...","[{'type': 'POOL SIZE', 'sq_ft': '288'}]",[]


# End Notebook